In [5]:
import torch
from torch_geometric.data import Data

# 예시 데이터: 노드 특성 (feature)와 엣지 인덱스 설정
node_features = torch.rand((180000, 10))  # 180,000개의 노드, 각 노드는 10개의 특성
edge_index = torch.randint(0, 180000, (2, 500000))  # 예시 엣지 인덱스 (500,000개의 엣지)

In [7]:
edge_index

tensor([[ 19309,  81334, 141500,  ..., 154973, 117791,  44683],
        [ 16248,  34634, 115243,  ...,  47101, 163301, 102108]])

In [ ]:
# 그래프 데이터 생성
data = Data(x=node_features, edge_index=edge_index)

In [2]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

In [8]:
from torch_geometric.loader import DataLoader

# 가상의 레이블 생성 (예: 노드 분류)
labels = torch.randint(0, 2, (180000,))  # 0 또는 1의 레이블

tensor([1, 1, 1,  ..., 0, 0, 1])

In [ ]:

# 데이터 로더 설정
train_loader = DataLoader([data], batch_size=1)

# 모델, 손실 함수, 옵티마이저 설정
model = GCN(input_dim=10, hidden_dim=16, output_dim=2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.CrossEntropyLoss()

# 학습 루프
for epoch in range(100):  # 100 에폭 학습
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index)
        loss = loss_fn(out, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

In [4]:
# 새로운 단일 데이터 생성
new_node_feature = torch.rand((1, 10))  # 새로운 노드의 특성

# 기존 그래프에 새로운 노드 추가
new_edge_index = torch.tensor([[0], [180000]])  # 새로운 노드와 기존 노드 연결 (예: 0번 노드와 연결)

# 엣지 인덱스 업데이트
updated_edge_index = torch.cat([data.edge_index, new_edge_index], dim=1)

# 새로운 데이터 생성
new_data = Data(x=torch.cat([data.x, new_node_feature], dim=0), edge_index=updated_edge_index)

# 예측 수행
model.eval()
with torch.no_grad():
    output = model(new_data.x, new_data.edge_index)
    prediction = output[-1]  # 새로운 노드의 예측
    predicted_class = prediction.argmax().item()  # 가장 높은 클래스 예측
    print(f'Predicted class for the new node: {predicted_class}')

Predicted class for the new node: 1
